In [1]:
%pip install -qU langchain_community beautifulsoup4 wikipedia arxiv pymupdf youtube-transcript-api python-dotenv langchain-chroma sentence-transformers langchain-groq

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 106.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.9 MB/s eta 0

In [2]:
%pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [3]:
from langchain_community.document_loaders import WebBaseLoader, WikipediaLoader, ArxivLoader, YoutubeLoader, PubMedLoader
from dotenv import load_dotenv
import os
import requests
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
def get_learning_preferences():
    print("Welcome to the Interactive Learning Assistant!\n")

    topic = input("What topic do you want to learn about? ")
    objective = input("What is your specific learning goal related to this topic? ")
    knowledge = input("What is your current level of knowledge on this topic? (e.g., beginner, intermediate, advanced) ")
    preferred_format = input("How do you prefer to learn? (text, videos, diagrams, examples, all) ")

    preferences = {
        "topic": topic.strip(),
        "learning_objective": objective.strip(),
        "prior_knowledge": knowledge.strip().lower(),
        "preferred_format": preferred_format.strip().lower()
    }

    print("\n Thanks! Here's a summary of your preferences:")
    for k, v in preferences.items():
        print(f" - {k.replace('_', ' ').capitalize()}: {v}")
    return preferences

user_preferences = get_learning_preferences()

Welcome to the Interactive Learning Assistant!

What topic do you want to learn about? neural networks
What is your specific learning goal related to this topic? back propagation
What is your current level of knowledge on this topic? (e.g., beginner, intermediate, advanced) beginner
How do you prefer to learn? (text, videos, diagrams, examples, all) text

 Thanks! Here's a summary of your preferences:
 - Topic: neural networks
 - Learning objective: back propagation
 - Prior knowledge: beginner
 - Preferred format: text


In [5]:
def preferences_to_query(preferences: dict) -> str:
    return (
        f"I'm a {preferences['prior_knowledge']} learner interested in '{preferences['topic']}'. "
        f"My goal is to learn about {preferences['learning_objective']}, and I prefer learning through {preferences['preferred_format']}."
    )
query = preferences_to_query(user_preferences)
query

"I'm a beginner learner interested in 'neural networks'. My goal is to learn about back propagation, and I prefer learning through text."

In [6]:
topic = user_preferences["topic"]

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [8]:
loader = WikipediaLoader(
    query= topic,
    load_max_docs=10,
    doc_content_chars_max=10000,
)
data1 = loader.load()
data1_split = text_splitter.split_documents(data1)
len(data1_split)

138

In [9]:
data1_split[0].page_content

'A neural network is a group of interconnected units called neurons that send signals to one another. Neurons can be either biological cells or mathematical models. While individual neurons are simple, many of them together in a network can perform complex tasks. There are two main types of neural networks.\n\nIn neuroscience, a biological neural network is a physical structure found in brains and complex nervous systems – a population of nerve cells connected by synapses.\nIn machine learning, an artificial neural network is a mathematical model used to approximate nonlinear functions. Artificial neural networks are used to solve artificial intelligence problems.\n\n\n== In biology =='

In [10]:
loader = ArxivLoader(
    query=topic,
    load_max_docs=10,
    doc_content_chars_max=20000,
    load_all_available_meta=False,

)
data2 = loader.load()
data2_split = text_splitter.split_documents(data2)
len(data2_split)

62

In [11]:
data2_split[6].page_content

'the axon to other cells.\nThere are\napproximately 86 billion neurons in\nthe human brain.\nImage modiﬁed\nfrom: https://www.smartsheet.com/\nneural-network-applications.\nFor a single neuron/node with input u ∈Rn, a mathematical model, named the perceptron\n[27], can be described as\ny = σ\n n\nX\ni=1\nWiui + b\n!\n= σ(W ⊤u + b),\n(1)\nwhere y is the activation of the neuron/node, Wi are the weights and b is the bias.\ny[0]\n1\ny[0]\n2\ny[0]\n3\ny[1]\n1\ninput\nlayer\noutput\nlayer\nFigure 2.\nSchematic representation of the perceptron\nwith a three dimensional input. For generality we denote\nthe input by y[0] = u and the output by y[1] = y. The\nweights Wi are applied on the arrows and the bias is added\nin the node y[1]\n1 .\nThe function σ : R →R is called activation function. Originally, in [27], it was proposed to\nchoose the Heaviside function as activation function to model whether a neuron ﬁres or not, i.e.\nσ(y) =\n(\n1\nif y ≥0,\n0\nif y < 0.'

In [12]:
from google.colab import userdata
api_key = userdata.get('YOUTUBE_API_KEY')

In [13]:
load_dotenv()
SEARCH_QUERY = topic
MAX_RESULTS = 5

# Changed API_KEY to api_key in the URL
url = f'https://www.googleapis.com/youtube/v3/search?part=snippet&type=video&q={SEARCH_QUERY}&maxResults={MAX_RESULTS}&key={api_key}'
response = requests.get(url)
data = response.json()

video_links = [
    f"https://www.youtube.com/watch?v={item['id']['videoId']}"
    for item in data['items']
]
video_links

['https://www.youtube.com/watch?v=jmmW0F0biz0',
 'https://www.youtube.com/watch?v=aircAruvnKk',
 'https://www.youtube.com/watch?v=kQl45ophSVQ',
 'https://www.youtube.com/watch?v=bfmFfD2RIcg',
 'https://www.youtube.com/watch?v=CqOfi41LfDw',
 'https://www.youtube.com/watch?v=Dbcx2_MO0LM',
 'https://www.youtube.com/watch?v=rEDzUT3ymw4',
 'https://www.youtube.com/watch?v=3TeEqgqPK8M',
 'https://www.youtube.com/watch?v=8hRujrzEfJk',
 'https://www.youtube.com/watch?v=NMZ0Tgc2jFQ']

In [14]:
data4 = []

for url in video_links:
    try:
        loader = YoutubeLoader.from_youtube_url(url, add_video_info=False)
        video_data = loader.load()
        data4.extend(video_data)
    except Exception as e:
        print(f"Skipping {url} due to error: {e}")

data4_split = text_splitter.split_documents(data4)
len(data4_split)

72

In [15]:
data4_split[0].page_content

"Here are five things to know about neural\xa0\nnetworks in under five minutes. Number one:\xa0\xa0 neural networks are composed of node layers. There\xa0\nis an input node layer, there is a hidden layer,\xa0\xa0 and there is an output layer. And these neural\xa0\nnetworks reflect the behavior of the human brain,\xa0\xa0 allowing computer programs to recognize patterns\xa0\nand solve common problems in the fields of AI and\xa0\xa0 deep learning. In fact, we should be describing\xa0\nthis as an artificial neural network, or an\xa0\xa0ANN, to distinguish it from the very un-artificial\xa0\nneural network that's operating in our heads. Now,\xa0\xa0 think of each node, or artificial neuron, as its own\xa0\nlinear regression model. That's number two.\xa0\xa0 Linear regression is a mathematical model that's\xa0\nused to predict future events. The weights of the\xa0\xa0 connections between the nodes determines how much\xa0\ninfluence each input has on the output. So each\xa0\xa0 node is compo

In [16]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={"device": "cuda"}
)

<ipython-input-16-6bfeb1e079e4>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="sample",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db"
)

In [51]:
document_ids1 = vector_store.add_documents(documents= data1_split)
document_ids2 = vector_store.add_documents(documents= data2_split)
document_ids3 = vector_store.add_documents(documents= data4_split)
document_ids = document_ids1 + document_ids2 + document_ids3

In [52]:
len(document_ids)

272

In [59]:
retriever = vector_store.as_retriever(search_type = "similarity", search_kwargs = {"k": 30})
retrieved_docs = retriever.invoke(query)
context = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [60]:
import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")
  print("API key set successfully!")

In [61]:
from langchain.chat_models import init_chat_model
llm = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

In [66]:
from langchain.prompts import PromptTemplate

template = """
You are an AI educational assistant. Based on the provided context and query, generate a detailed, well-structured educational report in **markdown** format with the following **exact** sections:

1. Title
Provide a concise, informative title.

2. Introduction
Introduce the topic, explain its importance.

3. Learning Objectives
List 3_4 bullet points on what learners will gain.

4. Concept Breakdown
Explain the core concepts in a logical flow with subheadings if needed.

5. Visual Aids Description
Describe 2–3 diagrams or figures that would help visualize key concepts.

6. Examples and Use Cases
Provide real-world examples, applications, or case studies.

7. Citations & References
Include citations or links to Wikipedia, arXiv, or YouTube that support the content.

8. Recommended Additional Resources
Suggest 3–5 resources (books, papers, or videos) for further study.

9. Conclusion and Next Steps
Summarize and recommend what to learn next.

Query: {query}

Educational Content:
{context}
"""

prompt = PromptTemplate(
    input_variables=["query", "context"],
    template=template
)

final_prompt = prompt.invoke({"query": query, "context": context})

In [67]:
response = llm.invoke(final_prompt)
print(response.content)

### Title
Introduction to Neural Networks: Understanding the Basics and Backpropagation

### Introduction
Neural networks are a fundamental concept in machine learning, inspired by the structure and function of the human brain. They are composed of interconnected nodes or neurons that process and transmit information. Neural networks have become a crucial tool in various fields, including image recognition, natural language processing, and decision-making. Understanding the basics of neural networks and how they learn through backpropagation is essential for working with these complex systems.

### Learning Objectives
* Understand the basic structure and function of neural networks
* Learn how neural networks process information through layers of interconnected neurons
* Familiarize yourself with the concept of backpropagation and its role in training neural networks
* Recognize the importance of neural networks in machine learning and artificial intelligence
* Understand how to apply 